# Epipolar Geometry

Draw epipoar line using essential matrix

E = [t]R

Steps
1. Select 3D points of an object in world coordinate
2. Create 2 cameras(store intrinsic and extrinsic matrix)
3. Convert World coordinate to pixel coordinate
4. Compute E
5. Generate epipolar line = E 0  point in camera frame

Each point in one camera generate corresponding epipolarline in another camera

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# ------------------ Step 1: Create 3D points ------------------
# Some random 3D points in world coordinates
points_3D = np.array([
    [1, 1, 5],
    [2, -1, 6],
    [-1, 2, 7],
    [0, -2, 8]
], dtype=float)

# ------------------ Step 2: Define camera intrinsics ------------------
K = np.array([[800, 0, 320],
              [0, 800, 240],
              [0,   0,   1]])  # simple pinhole camera

# Extrinsics for Camera 1 (Identity)
R1 = np.eye(3)
t1 = np.zeros((3, 1))

# Extrinsics for Camera 2 (translated + rotated)
R2, _ = cv2.Rodrigues(np.array([0.1, 0.2, 0.1]))  # small rotation
t2 = np.array([[1], [0], [0]])  # translated along x-axis

# Projection matrices
P1 = K @ np.hstack((R1, t1))
P2 = K @ np.hstack((R2, t2))

# ------------------ Step 3: Project 3D points into both cameras ------------------
def project_points(points, P):
    homog = np.hstack((points, np.ones((points.shape[0], 1))))
    img_pts = (P @ homog.T).T
    img_pts = img_pts[:, :2] / img_pts[:, 2][:, np.newaxis]
    return img_pts

pts1 = project_points(points_3D, P1)
pts2 = project_points(points_3D, P2)

# ------------------ Step 4: Compute Essential Matrix E = [t]_x R ------------------
# Relative motion (Camera2 wrt Camera1)
R = R2
T = t2.flatten()

# Skew-symmetric matrix of T
def skew(t):
    return np.array([[0, -t[2], t[1]],
                     [t[2], 0, -t[0]],
                     [-t[1], t[0], 0]])

E = skew(T) @ R
print("Essential Matrix:\n", E)

# ------------------ Step 5: Epipolar lines ------------------
# For each point in image1, compute corresponding epipolar line in image2
pts1_h = np.hstack((pts1, np.ones((pts1.shape[0], 1))))  # homogeneous
lines2 = (E @ pts1_h.T).T  # Epipolar lines in image2

# ------------------ Step 6: Plot ------------------
img_w, img_h = 640, 480

def draw_epipolar(img_pts, lines, ax, title):
    ax.set_title(title)
    ax.set_xlim(0, img_w)
    ax.set_ylim(img_h, 0)

    for pt, line in zip(img_pts, lines):
        x0, y0 = pt
        a, b, c = line

        # line: ax + by + c = 0 → y = -(a*x + c)/b
        x_vals = np.array([0, img_w])
        y_vals = -(a * x_vals + c) / b

        ax.plot(x_vals, y_vals, 'r-')
        ax.plot(x0, y0, 'bo')

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Left image (camera1 points)
ax[0].scatter(pts1[:,0], pts1[:,1], c='b')
ax[0].set_title("Camera 1 Image")
ax[0].set_xlim(0, img_w)
ax[0].set_ylim(img_h, 0)

# Right image (epipolar lines in camera2)
draw_epipolar(pts2, lines2, ax[1], "Camera 2 Image with Epipolar Lines")

plt.show()


error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'Rodrigues'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'
